# Android的人脸识别库 － 基于dlib库

## 创建工程向导

![](images/create_android_ndk_project_guide1.png)
![](images/create_android_ndk_project_guide2.png)
![](images/create_android_ndk_project_guide3.png)

## dlib库源代码添加到工程

* 下载[dlib库源代码](https://github.com/davisking/dlib)
* 把dlib目录下的dlib文件夹拷贝到app/src/main/

## 增加JNI接口

### 创建Java接口类
在app/src/main/java/com/wangjunjian/facerecognition下创建类FaceRecognition
```java
package com.wangjunjian.facerecognition;

import android.graphics.Rect;

public class FaceRecognition {
    static {
        System.loadLibrary("face-recognition");
    }

    public native void detect(String filename, Rect rect);

}
```

### 通过Java接口类输出C++头文件
打开Terminal窗口，输入命令(**Windows系统下要把:改为;**)
```bash
cd app/src/main/
javah -d jni -classpath /Users/wjj/Library/Android/sdk/platforms/android-21/android.jar:java com.wangjunjian.facerecognition.FaceRecognition
```

### 参考资料
* [JNI 无法确定Bitmap的签名](https://blog.csdn.net/wxxgreat/article/details/48030775)
* [在编辑JNI头文件的时候碰到无法确定Bitmap的签名问题](https://www.jianshu.com/p/b49bdcbfb5ed)

## 实现人脸检测
打开app/src/main/cpp/face-recognition.cpp
```cpp
#include <jni.h>
#include <string>
#include <dlib/image_processing/frontal_face_detector.h>
#include <dlib/image_io.h>

#include "jni/com_wangjunjian_facerecognition_FaceRecognition.h"

using namespace dlib;
using namespace std;

JNIEXPORT void JNICALL Java_com_wangjunjian_facerecognition_FaceRecognition_detect
        (JNIEnv *env, jobject clazz, jstring filename, jobject rect)
{
    const char* pfilename = env->GetStringUTFChars(filename, JNI_FALSE);

    static frontal_face_detector detector = get_frontal_face_detector();
    array2d<unsigned char> img;
    load_image(img, pfilename);

    env->ReleaseStringUTFChars(filename, pfilename);

    std::vector<rectangle> dets = detector(img, 0);

    if (dets.size() > 0)
    {
        rectangle faceRect = dets[0];

        jclass rectClass = env->GetObjectClass(rect);

        jfieldID fidLeft = env->GetFieldID(rectClass, "left", "I");
        env->SetIntField(rect, fidLeft, faceRect.left());
        jfieldID fidTop = env->GetFieldID(rectClass, "top", "I");
        env->SetIntField(rect, fidTop, faceRect.top());
        jfieldID fidRight = env->GetFieldID(rectClass, "right", "I");
        env->SetIntField(rect, fidRight, faceRect.right());
        jfieldID fidBottom = env->GetFieldID(rectClass, "bottom", "I");
        env->SetIntField(rect, fidBottom, faceRect.bottom());
    }
}
```

### 参考资料
*[Android使用JNI实现Java与C之间传递数据](https://blog.csdn.net/furongkang/article/details/6857610)

## 修改 app/CMakeLists.txt

```
# For more information about using CMake with Android Studio, read the
# documentation: https://d.android.com/studio/projects/add-native-code.html

# Sets the minimum version of CMake required to build the native library.

cmake_minimum_required(VERSION 3.4.1)

# 设置库输出路径变量
set(DISTRIBUTION_DIR ${CMAKE_SOURCE_DIR}/../distribution)

# 包含dlib的make信息
include(${CMAKE_SOURCE_DIR}/src/main/dlib/cmake)

# Creates and names a library, sets it as either STATIC
# or SHARED, and provides the relative paths to its source code.
# You can define multiple libraries, and CMake builds them for you.
# Gradle automatically packages shared libraries with your APK.

add_library( # Sets the name of the library.
             face-recognition

             # Sets the library as a shared library.
             SHARED

             # Provides a relative path to your source file(s).
             src/main/cpp/face-recognition.cpp )

# 设置每个平台的ABI输出路径
set_target_properties(face-recognition PROPERTIES
                      LIBRARY_OUTPUT_DIRECTORY
                      ${DISTRIBUTION_DIR}/libs/${ANDROID_ABI})

# Searches for a specified prebuilt library and stores the path as a
# variable. Because CMake includes system libraries in the search path by
# default, you only need to specify the name of the public NDK library
# you want to add. CMake verifies that the library exists before
# completing its build.

find_library( # Sets the name of the path variable.
              log-lib

              # Specifies the name of the NDK library that
              # you want CMake to locate.
              log )

# Specifies libraries CMake should link to your target library. You
# can link multiple libraries, such as libraries you define in this
# build script, prebuilt third-party libraries, or system libraries.

# 连接dlib和android
target_link_libraries( # Specifies the target library.
                       face-recognition
                       android
                       dlib

                       # Links the target library to the log library
                       # included in the NDK.
                       ${log-lib} )
```

### 参考资料
* [AndroidStudio用Cmake方式编译NDK代码](https://blog.csdn.net/joe544351900/article/details/53637549)

## 修改  app/build.gradle

```json
//修改为库
//apply plugin: 'com.android.application'
apply plugin: 'com.android.library'

android {
    compileSdkVersion 26
    defaultConfig {
        //移除应用ID
        //applicationId "com.wangjunjian.facerecognition"
        minSdkVersion 21
        targetSdkVersion 26
        versionCode 1
        versionName "1.0"
        testInstrumentationRunner "android.support.test.runner.AndroidJUnitRunner"
        externalNativeBuild {
            cmake {
                arguments '-DANDROID_PLATFORM=android-21',
                          '-DANDROID_TOOLCHAIN=clang', '-DANDROID_STL=c++_static', '-DCMAKE_BUILD_TYPE=Release ..'
                cppFlags "-frtti -fexceptions -std=c++11 -O3"
            }
        }
        //要生成的目标平台ABI
        ndk {
            abiFilters 'armeabi-v7a', 'arm64-v8a', 'x86', 'x86_64'
        }
    }
    buildTypes {
        release {
            minifyEnabled false
            proguardFiles getDefaultProguardFile('proguard-android.txt'), 'proguard-rules.pro'
        }
    }
    externalNativeBuild {
        cmake {
            path "CMakeLists.txt"
        }
    }
    //JNI库输出路径
    sourceSets {
        main {
            jniLibs.srcDirs = ['../distribution/libs']
        }
    }
    //消除错误 Caused by: com.android.builder.merge.DuplicateRelativeFileException: More than one file was found with OS independent path 'lib/x86/libface-recognition.so'
    packagingOptions {
        pickFirst 'lib/armeabi-v7a/libface-recognition.so'
        pickFirst 'lib/arm64-v8a/libface-recognition.so'
        pickFirst 'lib/x86/libface-recognition.so'
        pickFirst 'lib/x86_64/libface-recognition.so'
    }
}

//打包jar到指定路径
task makeJar(type: Copy) {
    delete 'build/libs/face-recognition.jar'
    from('build/intermediates/packaged-classes/release/')
    into('../distribution/libs/')
    include('classes.jar')
    rename('classes.jar', 'face-recognition.jar')
}
makeJar.dependsOn(build)

dependencies {
    implementation fileTree(dir: 'libs', include: ['*.jar'])
    implementation 'com.android.support:appcompat-v7:26.1.0'
    testImplementation 'junit:junit:4.12'
    androidTestImplementation 'com.android.support.test:runner:1.0.2'
    androidTestImplementation 'com.android.support.test.espresso:espresso-core:3.0.2'
}
```

### 参考资料
* [Android NDK samples with Android Studio](https://github.com/googlesamples/android-ndk)
* [Android studio 将 Module 打包成 Jar 包](https://www.cnblogs.com/xinaixia/p/7660173.html)
* [could not load library "libc++_shared.so" needed by "libgpg.so"](https://github.com/playgameservices/play-games-plugin-for-unity/issues/280)
* [Android NDK cannot load libc++_shared.so, gets "cannot locate symbol 'rand' reference](https://stackoverflow.com/questions/28504875/android-ndk-cannot-load-libc-shared-so-gets-cannot-locate-symbol-rand-refe)
* [记录Android-Studio遇到的各种坑](https://blog.csdn.net/u012874222/article/details/50616698)
* [Gradle flavors for android with custom source sets - what should the gradle files look like?](https://stackoverflow.com/questions/19461145/gradle-flavors-for-android-with-custom-source-sets-what-should-the-gradle-file)
* [Android Studio 2.2 gradle调用ndk-build](https://www.jianshu.com/p/0e50ae3c4d0d)
* [Android NDK: How to build for ARM64-v8a with minimumSdkVersion = 19](https://stackoverflow.com/questions/41102128/android-ndk-how-to-build-for-arm64-v8a-with-minimumsdkversion-19)

## 编译输出开发库

打开Terminal窗口，输入命令
```bash
./gradlew makeJar
```

![](images/create_android_ndk_project_guide4.png)

### 参考资料
* [-bash ：gradlew command not found](https://blog.csdn.net/yyh352091626/article/details/52343951)

## 查看jar文档列表
```bash
jar vtf distribution/libs/face-recognition.jar
```

### 参考资料
* [Linux环境下查看jar包的归档目录](https://blog.csdn.net/tanga842428/article/details/55101253)

## 参考资料
* [Face Landmarks In Your Android App](http://slides.com/boywang/face-landmarks-in-your-android-app/fullscreen#/)
* [dlib-android](https://github.com/tzutalin/dlib-android)
* [深入理解Android（一）：Gradle详解](http://www.infoq.com/cn/articles/android-in-depth-gradle/)
* [Android NDK Gradle3.0 以上最新生成.so之旅](https://blog.csdn.net/xiaozhu0922/article/details/78835144)
* [Android Studio 手把手教你NDK打包SO库文件，并提供对应API 使用它（赋demo）](https://blog.csdn.net/u011445031/article/details/72884703)
* [Building dlib for android ndk](https://stackoverflow.com/questions/41331400/building-dlib-for-android-ndk)
* [使用 Android Studio 写出第一个 NDK 程序（超详细）](https://blog.csdn.net/young_time/article/details/80346631)
* [Android studio3.0 JNI/NDK开发流程](https://www.jianshu.com/p/a37782b56770)
* [dlib 18 android编译dlib库，运行matrix_ex demo](https://blog.csdn.net/longji/article/details/78115807)
* [Android开发——Android Studio下使用Cmake在NDK环境下移植Dlib库](https://blog.csdn.net/u012525096/article/details/78950979)
* [android编译系统makefile(Android.mk)写法](http://www.cnblogs.com/hesiming/archive/2011/03/15/1984444.html)
* [dlib-android/jni/jni_detections/jni_pedestrian_det.cpp](https://github.com/tzutalin/dlib-android/blob/master/jni/jni_detections/jni_pedestrian_det.cpp)
* [Face Detection using MTCNN and TensorFlow in android](http://androidcodehub.com/face-detection-using-mtcnn-tensorflow-android/)